In [1]:
!pip install -q opencv-contrib-python
!pip install -q controlnet_aux
!pip install -q accelerate
!pip install -q torch
!pip install -q transformers
!pip install -q diffusers
!pip install -q gradio
!pip install -q xformers
!pip install -q segment-anything

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 80.1

In [2]:
from io import BytesIO
import IPython
import json
import os
from PIL import Image
import requests
import time
from google.colab import output

In [3]:
STABILITY_KEY = 'sk-5itUwtDqNblmSsl2bp9IZzDaTfvq41IL4b515zWLwY6qLnJo'

In [4]:
import numpy as np
import pandas as pd
import os
import math
from typing import List
from PIL import Image
from skimage import measure
import cv2
from google.colab.patches import cv2_imshow
import torch
import transformers
import accelerate
from diffusers import StableDiffusionControlNetPipeline
from diffusers import StableDiffusionInpaintPipeline
import gradio as gr
import torch
import cv2
import numpy as np
from PIL import Image
from diffusers import ControlNetModel, UniPCMultistepScheduler,StableDiffusionControlNetPipeline,DiffusionPipeline
from controlnet_aux import HEDdetector
from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel, UniPCMultistepScheduler
import sys
from xformers.ops import MemoryEfficientAttentionFlashAttentionOp
sys.path.append("..")
from controlnet_aux import NormalBaeDetector
from segment_anything import SamPredictor, sam_model_registry, SamAutomaticMaskGenerator
HOME = os.getcwd()
print("HOME:", HOME)
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
%cd {HOME}

!mkdir -p {HOME}/weights
%cd {HOME}/weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

%cd {HOME}
!git clone https://github.com/facebookresearch/segment-anything.git

%cd {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

%cd {HOME}
GROUNDING_DINO_CONFIG_PATH = os.path.join(
    HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
)
print(
    GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH)
)

GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(
    HOME, "weights", "groundingdino_swint_ogc.pth"
)
print(
    GROUNDING_DINO_CHECKPOINT_PATH,
    "; exist:",
    os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH),
)

SAM_CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(SAM_CHECKPOINT_PATH, "; exist:", os.path.isfile(SAM_CHECKPOINT_PATH))
%cd {HOME}/GroundingDINO
from groundingdino.util.inference import (
    load_model,
    load_image,
    predict,
    annotate,
    Model,
)

grounding_dino_model = Model(
    model_config_path=GROUNDING_DINO_CONFIG_PATH,
    model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH,
)
%cd {HOME}
# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
openpose = NormalBaeDetector.from_pretrained("lllyasviel/Annotators")
%cd {HOME}/GroundingDINO
model = load_model(
    "groundingdino/config/GroundingDINO_SwinT_OGC.py",
    "../weights/groundingdino_swint_ogc.pth",
)
BOX_THRESHOLD = 0.35
TEXT_THRESHOLD = 0.25
MODEL_TYPE = "vit_h"  # default
sam = sam_model_registry[MODEL_TYPE](checkpoint=SAM_CHECKPOINT_PATH).to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

sam_predictor = SamPredictor(sam)
def segment(
    sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray
) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(box=box, multimask_output=True)
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)
def generate_image_cnt(image, mask, prompt, pipe,control_image, seed):
    # resize for inpainting
    w, h = image.size
    in_image = image.resize((512, 512))
    in_mask = mask.resize((512, 512))
    control_image = control_image.resize((512, 512))

    generator = torch.Generator(device).manual_seed(seed)

    result = pipe(
        image=in_image,
        mask_image=in_mask,
        prompt=prompt,
        control_image = control_image,
        generator=generator,
    )
    del pipe
    result = result.images[0]

    return result.resize((w, h))
def send_generation_request(host,params):
    headers = {
        "Accept": "image/*",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    # Encode parameters
    files = {}
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    image_bytes = BytesIO()
    image.save(image_bytes, format='PNG')
    image_bytes = image_bytes.getvalue()
    mask_bytes = BytesIO()
    mask.save(mask_bytes, format='PNG')
    mask_bytes = mask_bytes.getvalue()
    if image is not None and image != '':
        files["image"]=image_bytes
    if mask is not None and mask != '':
        files["mask"]=mask_bytes
    if len(files)==0:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    return response
def stable_diffusion(image,mask,prompt,negative_prompt="",seed=0):
    image = image
    mask = mask
    prompt = prompt
    negative_prompt = negative_prompt
    seed = seed

    host = f"https://api.stability.ai/v2beta/stable-image/edit/inpaint"

    params = {
        "image" : image,
        "mask" : mask,
        "negative_prompt" : negative_prompt,
        "seed" : seed,
        "mode": "mask",
        "prompt" : prompt
    }

    response = send_generation_request(
        host,
        params
    )

    # Decode response
    output_image = response.content
    return output_image

/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


HOME: /content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 443, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 443 (delta 156), reused 138 (delta 129), pack-reused 233
Receiving objects: 100% (443/443), 12.86 MiB | 39.91 MiB/s, done.
Resolving deltas: 100% (228/228), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 6.0 MB/s eta 0:00:00
/content
/content/weights
/content
Cloning into 'segment-anything'...
remote: Enumerating objects: 295, done.
remote: Total 295 (delta 0), reused 0 (delta 0), pack-reused 295
Receiving objects: 100% (295/295), 18.30 MiB | 30.83 MiB/s, done.
Resolving deltas: 100% (155/155), done.
/content/weights
/content
/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True
/content/weights/gr

final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/content


scannet.pt:   0%|          | 0.00/291M [00:00<?, ?B/s]

Loading base model ()...Done.
Removing last two layers (global_pool & classifier).
/content/GroundingDINO
final text_encoder_type: bert-base-uncased


In [ ]:
def inpaint_image(edited_image, prompt):
    IMAGE_PATH = edited_image["composite"]
    IMAGE_NAME = IMAGE_PATH.split("/")[-1]
    image_bgr = cv2.imread(IMAGE_PATH)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    TEXT_PROMPT = "dress"
    detections, phrases = grounding_dino_model.predict_with_caption(
        image=image_bgr,
        caption=TEXT_PROMPT,
        box_threshold=BOX_THRESHOLD,
        text_threshold=TEXT_THRESHOLD,
    )
    detections.class_id = phrases
    detections.mask = segment(
        sam_predictor=sam_predictor, image=image_bgr, xyxy=detections.xyxy
    )
    mask_expanded = np.expand_dims(detections.mask[0], axis=-1)
    image = image_rgb*mask_expanded

    low_threshold = 25
    high_threshold = 100

    image = cv2.Canny(image, low_threshold, high_threshold)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    canny_image = Image.fromarray(image)
    fox = detections.mask[0]
    image_source_pil = Image.fromarray(image_rgb)
    image_mask_pil = Image.fromarray(fox)
    generate_image=stable_diffusion(
        image=image_source_pil,
        mask=image_mask_pil,
        prompt=prompt
    )
    if isinstance(generate_image, bytes):
        generate_image = Image.open(BytesIO(generate_image))

    return generate_image

# Gradio interface
def sketch_to_image_inpainting(image_editor, prompt):
    result = inpaint_image(image_editor, prompt)
    return result

iface = gr.Interface(
    fn=sketch_to_image_inpainting,
    inputs=[
        gr.ImageEditor(sources="upload",type='filepath', label="Draw on the image"),
        gr.Textbox(placeholder="Enter your prompt here...", label="Prompt")
    ],
    outputs=gr.Image(type="pil", label="Output Image"),
    title="Sketch to Image Inpainting with Stable Diffusion",
    description="Draw on the provided image and enter a prompt to generate an inpainted image using Stable Diffusion."
)

iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b67d518fed248a676c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
